In [11]:
import os
import requests
from bs4 import BeautifulSoup

BASE_PATH = os.path.dirname(os.getcwd())

In [70]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    #"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept-Encoding":"gzip, deflate",
    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT":"1","Connection":"close",
    "Upgrade-Insecure-Requests":"1"
}

In [2]:
publishers = [
    "pipoca+e+nanquim"
]

In [77]:
url = f"https://www.amazon.com.br/s?i=stripbooks&rh=p_30%3Apipoca+e+nanquim&s=relevanceexprank&Adv-Srch-Books-Submit.x=0&Adv-Srch-Books-Submit.y=0&unfiltered=1&ref=sr_adv_b"
#url = f"https://www.amazon.com.br/s?ie=UTF8&rh=p_30%3A{publishers[0]}&s=date-desc-rank"
req = requests.get(url, headers=headers)
soup = BeautifulSoup(req.content, "html.parser")

with open(f"{BASE_PATH}/data/temp.html", "w+") as file:
    file.write(str(soup.prettify()))

soup.find_all("div", {"class": "s-search-results"})

In [23]:
import os
import time
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By


BASE_PATH = os.path.dirname(os.getcwd())

In [12]:
publishers = [
    "pipoca+e+nanquim",
    "newpop",
    "jbc",
    "panini"
]

In [13]:
title_dataset = []
driver = webdriver.Chrome()

for publisher in publishers:
    # Acessa a página de busca utilizando query param
    url = f"https://www.amazon.com.br/s?ie=UTF8&rh=p_30%3A{publisher}&s=date-desc-rank"
    driver.get(url)
    time.sleep(2)

    while True:
        # Recupera lista de resultados
        elements = driver.find_element(By.CLASS_NAME, "s-result-list")
        books = elements.find_elements(By.XPATH, ".//div[@data-component-type='s-search-result']")

        # Percorre os livros encontrados e salva dados
        for book in books:
            row = {"publisher": " ".join(publisher.split("+")), "extract_at": date.today().isoformat()}

            book_link = book.find_element(By.XPATH, ".//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
            row["page_link"] = book_link.get_attribute("href")

            book_name = book.find_element(By.XPATH, ".//span[@class='a-size-base-plus a-color-base a-text-normal']")
            row["name"] = book_name.get_attribute("innerHTML")

            title_dataset.append(row)

        # Faz a troca de página, utilizando o botão de "próxima"
        old_url = driver.current_url
        driver.find_element(By.CLASS_NAME, "s-pagination-next").click()
        time.sleep(3)

        if driver.current_url == old_url:
            break

title_dataset = pd.DataFrame(title_dataset)
driver.close()

In [18]:
# Salva os dados primários para utilização posterior
title_dataset.to_csv(f"{BASE_PATH}/data/hqs/titles.csv", index=False)

In [39]:
title_dataset = pd.read_csv(f"{BASE_PATH}/data/hqs/titles.csv")
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    #"Accept-Encoding":"gzip, deflate",
    "Accept":"*/*",
    "Cookie": "i18n-prefs=BRL; session-id=140-7276983-7988533; session-id-time=2082787201l"
    #"DNT":"1","Connection":"close",
    #"Upgrade-Insecure-Requests":"1"
}

for i in range(0, len(title_dataset)):
    # Recupera dados da edição
    url = title_dataset["page_link"].values[i]
    name = title_dataset["name"].values[i].replace(":", " ").replace("?", "")

    # Acessa a página
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.content, "html.parser")

    # Salva o HTML para processamento futuro
    with open(f"{BASE_PATH}/data/hqs/download/{i} - {name}.html", "w+") as file:
        file.write(str(soup.prettify()))

    time.sleep(1)

KeyboardInterrupt: 